# TME 1 : Resolution de CSPs

Binome : Valentin COLLIARD et Madina TRAORE

In [1]:
from constraint import *

### Exemple 1 : Problème des 8 tours

In [2]:
# Dimension du problème
n = 8
# Création du problème
pb = Problem()
# Création d’une variable python de dimension n
cols = range(n)
# Création d’une variable cols dont le domaine est {1, ..., n}
pb.addVariables(cols, range(n))
# Ajout de la contrainte AllDiff
pb.addConstraint(AllDifferentConstraint())
# Récuperation de l’ensemble des solutions possibles
s = pb.getSolution()
print(s)

{0: 7, 1: 6, 2: 5, 3: 4, 4: 3, 5: 2, 6: 1, 7: 0}


### Exemple 2 : Problème du carré magique de dimension 3

In [3]:
# Dimension du problème
n = 3
# Creation du problème
p = Problem()
# Création d’une variable python représentant les nombres a placer dans la grille
x = range(1, n**2 + 1)
# Création d’une variable x dont le domaine est {1, ..., n**2 + 1}
p.addVariables(x, x)
# Ajout de la contrainte AllDiff
p.addConstraint(AllDifferentConstraint())
# Variable contenant la somme de chaque ligne/colonne/diagonale
s = n*n * (n*n + 1) / 6
# Ajout des contraintes du carré magique
for k in range(n):
    # ligne k
    p.addConstraint(ExactSumConstraint(s), [x[k*n+i] for i in range(n)])
    # colonne k
    p.addConstraint(ExactSumConstraint(s), [x[k+n*i] for i in range(n)])
    # premiere diagonale
    p.addConstraint(ExactSumConstraint(s), [x[n*i+i] for i in range(n)])
    # deuxieme diagonale
    p.addConstraint(ExactSumConstraint(s), [x[(n-1)*i] for i in range(1, n+1)])

s = p.getSolution()
print(s)

{5: 5, 1: 6, 9: 4, 3: 2, 7: 8, 4: 1, 8: 3, 2: 7, 6: 9}


### Exercice : Coloration de carte et CSP valués

#### Question 1

In [4]:
# Dimension du problème
n = 4
# Création du problème
P1 = Problem()
# Creation d’une variable python de dimension n
z = range(1,n+1)
# Création d’une variable z dont le domaine est {"rouge","vert","bleu"}
P1.addVariables(z, ["rouge","vert","bleu"])
# Ajout des contraintes de couleurs
P1.addConstraint(InSetConstraint(["rouge","bleu"]),[1])
P1.addConstraint(InSetConstraint(["bleu","vert"]),[2,3])
P1.addConstraint(InSetConstraint(["rouge"]),[4])

P1.addConstraint(lambda x, y : x != y, [1,2])
P1.addConstraint(lambda x, y : x != y, [1,3])
P1.addConstraint(lambda x, y : x != y, [1,4])
P1.addConstraint(lambda x, y : x != y, [2,3])
P1.addConstraint(lambda x, y : x != y, [3,4])

def solve_p1():
    return P1.getSolution()

print("Solution du problème P1 :", solve_p1())

Solution du problème P1 : None


Cette fonction retourne None. On en déduit que le problème P1 est trop contraignant, il n'admet aucune solution.

#### Question 2

In [5]:
# Dimension du problème
n = 4
# Création du problème
P2 = Problem()
# Création d’une variable python de dimension n
z = range(1,n+1)
# Création d’une variable z dont le domaine est {"rouge","vert","bleu"}
P2.addVariables(z, ["rouge","vert","bleu"])
# Ajout des contraintes de couleurs
P2.addConstraint(InSetConstraint(["rouge","vert"]),[1])
P2.addConstraint(InSetConstraint(["bleu","vert"]),[2,3])
P2.addConstraint(InSetConstraint(["vert"]),[4])

P2.addConstraint(lambda x, y : x != y, [1,2])
P2.addConstraint(lambda x, y : x != y, [1,3])
P2.addConstraint(lambda x, y : x != y, [1,4])
P2.addConstraint(lambda x, y : x != y, [2,3])
P2.addConstraint(lambda x, y : x != y, [3,4])

def solve_p2():
    return P2.getSolution()

print("Solution du problème P2 :",solve_p2())

Solution du problème P2 : {1: 'rouge', 3: 'bleu', 2: 'vert', 4: 'vert'}


Cette fonction retourne {1: 'rouge', 2: 'vert', 3: 'bleu', 4: 'vert'}. On en déduit que le problème P2 est une réduction du problème initial permettant de trouver une solution a celui-ci.

#### Question 3

In [23]:
from copy import *

class Node:
    
    def __init__(self,pb,var,u):
        self.pb = pb
        self.var = var
        self.u = u
        self.T = u
        self.fils = []
        
# Dimension du problème
n = 4
# Création du problème
P = Problem()
# Création d’une variable python de dimension n
z = range(1,n+1)
# Creation d’une variable z dont le domaine est {"rouge","vert","bleu"}
P.addVariables(z, ["rouge","vert","bleu"])

P.addConstraint(lambda x, y : x != y, [1,2])
P.addConstraint(lambda x, y : x != y, [1,3])
P.addConstraint(lambda x, y : x != y, [1,4])
P.addConstraint(lambda x, y : x != y, [2,3])
P.addConstraint(lambda x, y : x != y, [3,4])

U = dict()
U[1] = {"bleu" : 1, "vert" : 0.3, "rouge" : 0.8}
U[2] = {"bleu" : 0.2, "vert" : 0.6, "rouge" : 1}
U[3] = {"bleu" : 1, "vert" : 0.7, "rouge" : 0.4}
U[4] = {"bleu" : 0.5, "vert" : 1, "rouge" : 0.9}

def branch_and_bound(noeud,noeuds_ouverts,T_noeuds_ouverts):
    solutions = noeud.pb.getSolutions() #permet de récupérer les colorations respectant les contraintes
    
    # Ajout dans col_poss les couleurs possibles pour la zone que l'on est en train de traiter
    col_poss = []
    for sol in solutions:
        for key,val in sol.items():
            if key == noeud.var:
                col_poss.append(val)
    
    # Construction de l'arbre des colorations possibles
    for col, ut in U[noeud.var].items():
        if col in col_poss:
            pb = deepcopy(noeud.pb)
            pb.addConstraint(InSetConstraint(col),[noeud.var])
            f = Node(pb,noeud.var+1,U[noeud.var][col])
            if noeud.T < f.T:
                f.T = noeud.T
            noeud.fils.append(f)
    
    T_fils = [f.T for f in noeud.fils]
    T_noeuds_ouverts += T_fils
    T_noeuds_ouverts = sorted(T_noeuds_ouverts) # liste des utilités données par la fonction T de tous les noeuds ouverts 
    
    for T in T_noeuds_ouverts:
        for f in noeud.fils:
            if T == f.T:
                noeuds_ouverts.append(f) # liste des noeuds ouverts
    
    print("\nUtilités des noeuds ouverts :",T_noeuds_ouverts)
                
    T_max_cour = T_noeuds_ouverts.pop() #utilité maximale des noeuds encore ouverts
    
    next_noeud = None
    for i in range(len(noeuds_ouverts)):
        if noeuds_ouverts[i].T == T_max_cour:
            next_noeud = noeuds_ouverts[i]
            del noeuds_ouverts[i]
            break
            
    if next_noeud: # s'il y a encore un noeud à explorer
        print("Utilité du prochain noeud à explorer :",next_noeud.u)
        if next_noeud.var < 5: # si on n'est pas sur une feuille de l'arbre
            return branch_and_bound(next_noeud,noeuds_ouverts,T_noeuds_ouverts)
        else:
            print("Une solution optimale a été trouvée.")
            u_S = next_noeud.T
    else:
        print("Pas de meilleur noeud à explorer.")
    return noeud.pb.getSolution(),u_S

S,u_S = branch_and_bound(Node(P,1,1),[],[])

print("\nSolution du problème P :",S)

print("\nUtilité de la solution trouvée :",u_S)   


Utilités des noeuds ouverts : [0.3, 0.8, 1]
Utilité du prochain noeud à explorer : 1

Utilités des noeuds ouverts : [0.3, 0.6, 0.8, 1]
Utilité du prochain noeud à explorer : 1

Utilités des noeuds ouverts : [0.3, 0.6, 0.7, 0.8]
Utilité du prochain noeud à explorer : 0.8

Utilités des noeuds ouverts : [0.2, 0.3, 0.6, 0.6, 0.7]
Utilité du prochain noeud à explorer : 0.7

Utilités des noeuds ouverts : [0.2, 0.3, 0.6, 0.6, 0.7]
Utilité du prochain noeud à explorer : 0.9
Une solution optimale a été trouvée.

Solution du problème P : {1: 'bleu', 3: 'vert', 2: 'rouge', 4: 'rouge'}

Utilité de la solution trouvée : 0.7


La solution retournée par l'algorithme est la même que celle trouvée avec le problème réduit P2 : {1: 'bleu', 3: 'vert', 2: 'rouge', 4: 'rouge'}.

#### Question 4

In [25]:
from copy import *

class Node:
    
    def __init__(self,pb,var,u):
        self.pb = pb
        self.var = var
        self.u = u
        self.T = u
        self.fils = []
        

# Dimension du probleme
n = 4
# Creation du probleme (colorier la figure en attribuant une couleur (rouge, 
# vert ou bleu a chaque zone) de sorte que deux zones voisines soient de
# couleur differentes)
P = Problem()
# Creation d’une variable python de dimension n
z = range(1,n+1)
# Creation d’une variable z dont le domaine est {"rouge","vert","bleu"}
P.addVariables(z, ["rouge","vert","bleu"])

P.addConstraint(lambda x, y : x != y, [1,2])
P.addConstraint(lambda x, y : x != y, [1,3])
P.addConstraint(lambda x, y : x != y, [1,4])
P.addConstraint(lambda x, y : x != y, [2,3])
P.addConstraint(lambda x, y : x != y, [3,4])

U = dict()
U[1] = {"bleu" : 1, "vert" : 0.3, "rouge" : 0.8}
U[2] = {"bleu" : 0.2, "vert" : 0.6, "rouge" : 1}
U[3] = {"bleu" : 1, "vert" : 0.7, "rouge" : 0.4}
U[4] = {"bleu" : 0.5, "vert" : 1, "rouge" : 0.9}

s = dict()
u = dict()
liste_u = [] #liste triée des utilités rencontrées

def branch_and_bound(noeud,noeuds_ouverts,T_noeuds_ouverts):
    solutions = noeud.pb.getSolutions()
    
    # Ajout dans col_poss les couleurs possibles pour la zone que l'on est en train de traiter
    col_poss = []
    for sol in solutions:
        for key,val in sol.items():
            if key == noeud.var:
                col_poss.append(val)
    
    # Construction de l'arbre des colorations possibles
    for col, ut in U[noeud.var].items():
        if col in col_poss:
            pb = deepcopy(noeud.pb)
            pb.addConstraint(InSetConstraint(col),[noeud.var])
            f = Node(pb,noeud.var+1,U[noeud.var][col])
            f.T *= noeud.T
            noeud.fils.append(f)
    
    T_fils = [f.T for f in noeud.fils]
    T_noeuds_ouverts += T_fils
    T_noeuds_ouverts = sorted(T_noeuds_ouverts) # liste des utilités données par la fonction T de tous les noeuds ouverts
    
    for T in T_noeuds_ouverts:
        for f in noeud.fils:
            if T == f.T:
                noeuds_ouverts.append(f) # liste des noeuds ouverts
                
    print("Utilités des noeuds ouverts :",T_noeuds_ouverts)
            
    T_max_cour = T_noeuds_ouverts.pop()  #utilité maximale des noeuds encore ouverts
    
    
    for i in range(len(noeuds_ouverts)):
        if noeuds_ouverts[i].T == T_max_cour:
            next_noeud = noeuds_ouverts[i]
            del noeuds_ouverts[i]
            break
            
    if next_noeud:
        print("Utilité du prochain noeud à explorer :",next_noeud.u)
        if next_noeud.var < 5:
            return branch_and_bound(next_noeud,noeuds_ouverts,T_noeuds_ouverts)
        else:
            print("Une solution optimale a été trouvée.")
            u_S = next_noeud.T
    else:
        print("Pas de meilleur noeud à explorer.")

    return noeud.pb.getSolution(),u_S

S,u_S = branch_and_bound(Node(P,1,1),[],[])

print("\nSolution du problème P :",S)

u = dict()
for key,value in S.items():
    u[key] = U[key][value]
print("\nUtilité de chaque zone de la solution trouvée :",u)

print("\nUtilité de la solution trouvée :",u_S)    

Utilités des noeuds ouverts : [0.3, 0.8, 1]
Utilité du prochain noeud à explorer : 1
Utilités des noeuds ouverts : [0.3, 0.6, 0.8, 1]
Utilité du prochain noeud à explorer : 1
Utilités des noeuds ouverts : [0.3, 0.6, 0.7, 0.8]
Utilité du prochain noeud à explorer : 0.8
Utilités des noeuds ouverts : [0.16000000000000003, 0.3, 0.48, 0.6, 0.7]
Utilité du prochain noeud à explorer : 0.7
Utilités des noeuds ouverts : [0.16000000000000003, 0.3, 0.48, 0.6, 0.63]
Utilité du prochain noeud à explorer : 0.9
Une solution optimale a été trouvée.

Solution du problème P : {1: 'bleu', 3: 'vert', 2: 'rouge', 4: 'rouge'}

Utilité de chaque zone de la solution trouvée : {1: 1, 3: 0.7, 2: 1, 4: 0.9}

Utilité de la solution trouvée : 0.63


Utiliser T(x, y) = min{x, y} permet seulement de maximiser l'utilité minimale de la coloration : de cette façon, on fait seulement en sorte que la zone la "moins satisfaite" par sa coloration soit la moins insatisfaite possible, on ne prend réellement en compte qu'une seule zone. En utilisant T(x, y) = x x y au lieu de T(x, y) = min{x, y}, on cherche à avoir une "satisfaction globale" optimale.

Exemple :

Soient les colorations C1 = {1: 'vert', 2: 'bleu', 3: 'rouge', 4: 'bleu'} et C2 = {1: 'rouge', 2: 'bleu', 3: 'vert', 4: 'bleu'}. Les utilités correspondantes sont U1 = {1: 0.3, 2: 0.2, 3: 0.4, 4: 0.5} et U2 = {1: 0.8, 2: 0.2, 3: 0.7, 4: 0.5}.

Si l'on utilise T(x, y) = min{x, y}, on a f_T(U1) = f_T(U2) = 0.2. La fonction T ne permet donc pas de distinguer ces 2 solutions bien que la coloration C2 paraisse plus avatageuse pour l'esemble des zones que la coloration C1.

Si l'on utilise T(x, y) = x x y, on a f_T(U1) = 0.3x0.2x0.4x0.5 = 0.012 et f_T(U2) = 0.8x0.2x0.7x0.5 = 0.056.
f_T(U2) > f_T(U1) donc cette fonction T traduit bien le fait que la coloration C2 est plus avatageuse pour l'ensemble des zones que la coloration C1.